# Hands on Workshop Building a Chat App With Memory using Seldon

This demo implements a seldon-core-v2 pipeline with integrated state using the memory rt and a choice of two LLM backends: the OpenAI RT or the LocalLLM RT. This is a demo of the following seldon products:

1. MLserver memory runtime
2. MLserver openai runtime
3. MLserver local runtime


## Being Deplyed today:
    - Two memory components
    - A local chat RT component
    - A chat pipeline app use the LocalLLM RT

In order to run a terminal interface with the app use:

```sh
python chat.py --target=<target> --memory_id=<memory-id>
```

where memory-id is the id of a converstation and is optional (Not sepcifying will result in a new memory_id and converstation). And target is one of local or openai and specifies which RT to talk to.


To remove use:

```
make undeploy
```

## SCV2 flow

### Chat App Flow

The chat app pipeline looks like:

```mermaid

flowchart LR
    input([input])
    output([output])
    filesys[(FILE SYSTEM)]
    memory_1
    memory_2
    OAI["MLSERVER OAI"]

    input --> memory_1 --> OAI --> output
    filesys <--> memory_1
    memory_2 --> filesys
    OAI --> memory_2

```

We start by creating our directorys in which we will put our model configurations inb

In [ ]:
!mkdir ./models
!mkdir ./models/memory
!mkdir ./models/local-models

## Create the Memory Component Model

We will be creating a `model-settings.json` for our memory component to be used within our chat pipeline.

The memory component is used to store the chat history and give the flexibility throughout a Seldon Core v2 pipeline to access the memory of that chat.

In [ ]:
%%writefile models/memory/model-settings.json
{
    "name": "conversational_memory",
    "implementation": "mlserver_memory.ConversationalMemory",
    "parameters": {
        "extra": {
            "database_config": {
                "database": "filesys"
            },
            "memory_config": {
                "window_size": 10,
                "tensor_names": ["content", "role"]
            }
        }
    }
}

Now its time to confgiure the LLM model itself.  

The Local LLM runtime can support three libraries  

- Transformers
- vLLM
- Deepspeed

We will be using the transformers library for this example and include a prompt template.
For the model confiugration there will be two files being created `model-settings.json` and `prompt.jinja`

In [ ]:
%%writefile models/local-models/model-settings.json
{
  "name": "gpt2",
  "implementation": "mlserver_llm_local.runtime.Local",
  "parameters": {
    "uri": "gpt2",
    "extra": {
      "backend": "transformers",
      "model": {
        "enable_profile": "False",
        "device": "cpu"
      },
      "prompt": {
        "uri": "./prompt.jinja",
        "enable": "True",
        "tokens": {
          "eos_token": "<|endoftext|>"
        }
      }
    }
  }
}

And now the prompt file

In [ ]:
%%writefile models/local-models/prompt.jinja
{% for message in messages %}
  {{ message['content'].strip() }}
{% endfor %}

Now that the model settings are ready to go we will upload them to a google storage bucket to be used Model Deployment configurations in the LLM runtimes of Seldon Core v2

In the provided `upload_models.py` file please update line 14 with your name

```
blob = bucket.blob(f'llm/[YOUR NAME]/chat-memory/{directory}/{file}')
```

In [31]:
NAME = "josh-test"

In [32]:
import subprocess

# Running the script with the name variable as an argument
result = subprocess.run(['python', 'upload_models.py', NAME], capture_output=True, text=True)

# Print output and error, if any
print("Output:", result.stdout)
print("Error:", result.stderr)


Output: Uploading memory/model-settings.json
Uploading local-model/prompt.jinja
Uploading local-model/model-settings.json

Error: 


## Deploy the Chat Application pipeline into Seldon Core v2 using the LLM Module

It is now time to deploy our models to kubernetes and run our chat application. 

When deploying use cases into production with Seldon Core v2 there is a two pronged workflow in order to leverage the reusability of deploy machine learning models

Step 1: Deploy models
Step 2: Deploy Pipelines

We will setup the `deployments` directory to keep things organized

In [33]:
!mkdir deployments

mkdir: deployments: File exists


### Step 1: Deploy Models

Deploying LLMs in Seldon is similar to deploy traditional models as well. We create a manifest file containg the model configurations.

This manifest is composed of Seldon's `model` CRD.

For our use case we will be registering 3 seperate models while reusing the same configuration for both the combine-question and combine-answer models registered in Seldon Core 

The Pipeline that we will configure looks as below:

Step 1: Take input query and memory for context of chat
Step 2: Take combined content and process with LLM
Step 3: Append LLM output to memory

#### We create the models manifest to deploy the models into kubernetes

In [34]:
import yaml

models_manifest = f"""
apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: combine-question
spec:
  storageUri: "gs://josh-seldon/llm/chat-memory/{NAME}/memory"
  requirements:
  - memory
---
apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: combine-answer
spec:
  storageUri: "gs://josh-seldon/llm/chat-memory/{NAME}/memory"
  requirements:
  - memory
---
apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: localgpt
spec:
  storageUri: "gs://josh-seldon/llm/chat-memory/{NAME}/local-model"
  requirements:
  - llm-local
"""

# Load multiple documents
models = list(yaml.safe_load_all(models_manifest))

# Save each document separately in the YAML file
with open('deployments/models.yaml', 'w') as file:
    yaml.safe_dump_all(models, file)



And Now we apply the Models manifest to the Namespace we will be deploying our use case

In [35]:
!kubectl apply -f deployments/models.yaml -n seldon
!kubectl wait --for condition=ready --timeout=300s model --all -n seldon

model.mlops.seldon.io/combine-question created
model.mlops.seldon.io/combine-answer created
model.mlops.seldon.io/localgpt created
model.mlops.seldon.io/combine-answer condition met
model.mlops.seldon.io/combine-question condition met
model.mlops.seldon.io/localgpt condition met


In [30]:
!kubectl delete -f deployments/models.yaml -n seldon
!kubectl wait --for condition=ready --timeout=300s model --all -n seldon

model.mlops.seldon.io "combine-question" deleted
model.mlops.seldon.io "combine-answer" deleted
model.mlops.seldon.io "localgpt" deleted
error: no matching resources found


## TODO: Add test prediction to LLM

#### Now we create the Seldon Core Pipeline to tie everything together for our chat application

In [40]:
import yaml

pipeline_manifest = f"""
apiVersion: mlops.seldon.io/v1alpha1
kind: Pipeline
metadata:
  name: local-chat-memory
spec:
  steps:
    - name: combine-question
      inputs:
      - local-chat-memory.inputs.memory_id
      - local-chat-memory.inputs.role
      - local-chat-memory.inputs.content
    - name: localgpt
      inputs:
      - combine-question.outputs.role
      - combine-question.outputs.content
    - name: combine-answer
      inputs:
      - local-chat-memory.inputs.memory_id
      - localgpt.outputs.role
      - localgpt.outputs.content
  output:
    steps:
    - localgpt
"""

# Load multiple documents
pipeline = list(yaml.safe_load_all(pipeline_manifest))

# Save each document separately in the YAML file
with open('deployments/pipeline.yaml', 'w') as file:
    yaml.safe_dump_all(pipeline, file)



In [41]:
!kubectl apply -f deployments/pipeline.yaml -n seldon
!kubectl wait --for condition=ready --timeout=300s pipeline --all -n seldon

pipeline.mlops.seldon.io/local-chat-memory created
pipeline.mlops.seldon.io/local-chat-memory condition met
